In [1]:
import scrapy

In [2]:
https://franchise.ftc.go.kr/mnu/00013/program/userRqst/list.do
?searchCondition=&searchKeyword=&column=brd&selUpjong=21&selIndus=&pageUnit=300&pageIndex=4

In [13]:
import requests
from bs4 import BeautifulSoup


resp = requests.get(
    "https://franchise.ftc.go.kr/mnu/00013/program/userRqst/list.do",
    params = {
        "searchCondition" : "",
        "searchKeyword" : "",
        "column" : "brd",
        "selUpjong":"21",
        "selIndus" : "",
        "pageUnit" : "300",
        "pageIndex" : "4"
    }
)

In [12]:
#print(resp.content.decode())
print(resp.url)

https://franchise.ftc.go.kr/mnu/00013/program/userRqst/list.do?searchCondition=&searchKeyword=&column=brd&selUpjong=21&selIndus=&pageUnit=300&pageIndex=4
